In [14]:
import os
from medAI.modeling import create_model 
import cv2
from PIL import Image
import numpy as np 
import torch

model = create_model('micro_seg_net')
model.eval()

def run_on_single_image(path):
    
    from PIL import Image 
    image = Image.open(path)
    size = image.size

    image = np.array(image)
    image = image / 255 
    image = cv2.resize(image, (224, 224), interpolation=cv2.INTER_LINEAR)

    input = torch.from_numpy(image).unsqueeze(0).unsqueeze(0).float()

    outputs = model(input).sigmoid() 
    #print(outputs.shape)

    #import matplotlib.pyplot as plt
    #plt.imshow(input[0][0].detach().cpu().numpy(), cmap='gray')
    #plt.contour(outputs[0][0] > 0.5)
    #plt.figure()

    outputs = (outputs[0][0] > 0.5).numpy().astype('uint8')
    save_path = os.path.join(
            os.path.dirname(path), 
            'micro_seg_net_prostate_mask.png'
    )
    Image.fromarray(outputs).convert('L').resize(size, Image.Resampling.NEAREST).save(
        os.path.join(
            os.path.dirname(path), 
            'micro_seg_net_prostate_mask.png'
        )
    )

run_on_single_image('/h/pwilson/projects/medAI/data/OPTIMUM/processed/UA_annotated_needles/UA-078/UA-078-005/image.png')

#plt.imshow(outputs[0][0].detach().cpu().numpy(), cmap='gray', alpha=0.5)

/fs01/home/pwilson/projects/medAI/medAI/modeling/micro_seg_net/micro_seg_net.py:50: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(checkpoint

In [20]:
from glob import glob 
all_paths = list(glob(
    "/h/pwilson/projects/medAI/data/OPTIMUM/processed/UA_annotated_needles/**/image.png", recursive=True
))

from tqdm import tqdm 
for path in tqdm(all_paths): 
    run_on_single_image(path)

100%|██████████| 1041/1041 [03:17<00:00,  5.26it/s]


In [7]:
outputs.shape

torch.Size([1, 1, 224, 224])